In [1]:
import sys
from pathlib import Path

In [2]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

2022-11-13 11:58:49.615092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 11:58:49.786512: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 11:58:50.277148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2022-11-13 11:58:50.277210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: can

In [3]:
sys.path.append(str(Path("../../ssl").resolve()))

In [4]:
from src.models.pi_model.pi_model import PiModel
from src.models.pi_model.pi_model_config import PiModelConfig
from src.trainers.basic.categorical_ce import CategoricalCETrainer
from src.trainers.basic.categorical_ce_config import CategoricalCETrainerConfig
from src.data_loaders.basic.categorical_ce_config import CategoricalCEDataLoaderConfig
from src.data_loaders.basic.categorical_ce import CategoricalCEDataLoader

# Description

In this notebook, the AlexNet model will be trained on the CIFAR-10 dataset using only 25% of the labelled dataset.

## Set up Experiment

In [5]:
class TrainerConfig(CategoricalCETrainerConfig):
    num_epochs = 200

train_config = TrainerConfig()

In [6]:
class ModelConfig(PiModelConfig):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [7]:
class DataLoaderConfig(CategoricalCEDataLoaderConfig):
    batch_size = 64
    num_classes = 10
    shuffle_buffer_size = 5000 # dataset size    
    
    # switch augmenters off
    blur_params = None
    crop_params = None
    jitter_params = None

data_loader_config = DataLoaderConfig()

## Get Datasets

In [8]:
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.cifar10.load_data()

In [9]:
# get train dataset by subsampling 10% of the full training dataset (stratified by labels)
# add the rest as unlabelled samples
x_train_unlabelled, x_train_labelled, y_train_unlabelled_, y_train_labelled = train_test_split(
    x_train_full, y_train_full,
    stratify = y_train_full,
    test_size = 0.10, random_state = 42
)

# cast types
y_train_unlabelled = (-1 * np.ones_like(y_train_unlabelled_)).astype(np.int64)
y_train_labelled = y_train_labelled.astype(np.int64)
y_test_full = y_test_full.astype(np.int64)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((x_train_labelled, y_train_labelled))
val_data = tf.data.Dataset.from_tensor_slices((x_test_full, y_test_full))

2022-11-13 11:58:52.157490: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 11:58:52.163400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 11:58:52.163887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 11:58:52.164777: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
# create train dataset
train_data = CategoricalCEDataLoader(train_data, data_loader_config)(training = True)

In [12]:
# create test dataset
val_data = CategoricalCEDataLoader(val_data, data_loader_config)(training = False)

In [13]:
print(f"Train dataset size: {train_data.cardinality()}")
print(f"Validation dataset size: {val_data.cardinality()}")

Train dataset size: 79
Validation dataset size: 157


# Train Model

In [14]:
model = PiModel(model_config)()

In [15]:
trainer = CategoricalCETrainer(
    model, train_data, train_config,
    val_dataset = val_data)

In [16]:
trainer.train()

2022-11-13 11:58:53.598286: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


Training loss at epoch 0 is : 2.27. Validation loss is : 2.14. Validation acc. is : 18.05.
Training loss at epoch 1 is : 2.08. Validation loss is : 1.98. Validation acc. is : 25.71.
Training loss at epoch 2 is : 1.93. Validation loss is : 1.89. Validation acc. is : 26.80.
Training loss at epoch 3 is : 1.83. Validation loss is : 1.75. Validation acc. is : 33.30.
Training loss at epoch 4 is : 1.79. Validation loss is : 1.72. Validation acc. is : 35.58.
Training loss at epoch 5 is : 1.75. Validation loss is : 1.71. Validation acc. is : 35.68.
Training loss at epoch 6 is : 1.70. Validation loss is : 1.67. Validation acc. is : 37.21.
Training loss at epoch 7 is : 1.69. Validation loss is : 1.63. Validation acc. is : 38.90.
Training loss at epoch 8 is : 1.63. Validation loss is : 1.62. Validation acc. is : 39.51.
Training loss at epoch 9 is : 1.60. Validation loss is : 1.61. Validation acc. is : 40.36.
Training loss at epoch 10 is : 1.58. Validation loss is : 1.64. Validation acc. is : 39.47